In [1]:
import numpy as np
import csv
import time
import psutil
import os
from quantum_serverless import ServerlessProvider
from qiskit.circuit.library import EfficientSU2
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from qiskit_algorithms.utils import algorithm_globals
from quantum_serverless import distribute_program
import pandas as pd
from sklearn.datasets import load_iris

provider = ServerlessProvider(
    username=os.environ.get("GATEWAY_USER", "user"),
    password=os.environ.get("GATEWAY_PASSWORD", "password123"),
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
)

# Function to get CPU and memory usage
def get_cpu_memory_usage():
    cpu_percent = psutil.cpu_percent()
    mem_percent = psutil.virtual_memory().percent
    return cpu_percent, mem_percent

# Function to calculate mean CPU and memory usage
def calculate_mean_usage():
    cpu_values = []  # List to store CPU usage values
    mem_values = []  # List to store memory usage values

    for _ in range(5):  # Adjust the number of samples as needed
        cpu_usage, mem_usage = get_cpu_memory_usage()
        cpu_values.append(cpu_usage)
        mem_values.append(mem_usage)
        time.sleep(2)

    mean_cpu = sum(cpu_values) / len(cpu_values)
    mean_mem = sum(mem_values) / len(mem_values)

    return mean_cpu, mean_mem


def prepare_data():
    iris_data = load_iris()
    features = iris_data.data
    labels = iris_data.target
    features = MinMaxScaler().fit_transform(features)
    df = pd.DataFrame(features, columns=iris_data.feature_names)
    df["class"] = pd.Series(labels)

    features = df.drop("class", axis=1).values
    labels = df["class"].values

    return train_test_split(
        features, labels, train_size=0.8, random_state=algorithm_globals.random_seed
    )

@distribute_program(provider)
def run_quantum_program(num_qubits, reps, iterations, train_features, train_labels, test_features, test_labels):
    from qiskit_machine_learning.algorithms.classifiers import VQC
    from qiskit.algorithms.optimizers import COBYLA
    from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
    from qiskit import Aer
    from qiskit.utils import QuantumInstance

    feature_map = ZZFeatureMap(feature_dimension=num_qubits, reps=1)
    ansatz = RealAmplitudes(num_qubits=num_qubits, reps=reps)
    optimizer = COBYLA(maxiter=iterations)

    backend = Aer.get_backend("qasm_simulator")
    quantum_instance = QuantumInstance(backend)

    vqc = VQC(
        feature_map=feature_map,
        ansatz=ansatz,
        optimizer=optimizer,
        quantum_instance=quantum_instance,
    )

   
    vqc.fit(train_features, train_labels)

    train_score = vqc.score(train_features, train_labels)
    test_score = vqc.score(test_features, test_labels)

    return {"Train_Score": train_score, "Test_Score": test_score}

train_features, test_features, train_labels, test_labels = prepare_data()

num_qubits_values = [2, 4, 6]
reps_values = [1, 2, 4, 6]
iterations_list = [10, 50, 100]


csv_file_path = "metrics.csv"
with open(csv_file_path, mode='w', newline='') as csv_file:
    fieldnames = ["Num_Qubits", "Reps", "Iterations", "Mean_CPU", "Mean_Memory", "Elapsed_Time"]
    csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    csv_writer.writeheader()

    for num_qubits in num_qubits_values:
        for reps_value in reps_values:
            for iterations in iterations_list:
                ansatz = EfficientSU2(num_qubits, reps=reps_value)
                print(f"\nQuantum Circuit for {num_qubits} Qubits, {reps_value} Layers, and {iterations} Iterations:")
                print(ansatz.decompose().draw())

                mean_cpu_before, mean_mem_before = calculate_mean_usage()
                start_time_total = time.time()

                job_result = run_quantum_program(
                    num_qubits=num_qubits,
                    reps=reps_value,
                    iterations=iterations,
                    train_features=train_features,
                    train_labels=train_labels,
                    test_features=test_features,
                    test_labels=test_labels
                ).result()

                mean_cpu_after, mean_mem_after = calculate_mean_usage()
                elapsed_time_total = time.time() - start_time_total

                csv_writer.writerow({
                    "Num_Qubits": num_qubits,
                    "Reps": reps_value,
                    "Iterations": iterations,
                    "Mean_CPU": (mean_cpu_before + mean_cpu_after) / 2,
                    "Mean_Memory": (mean_mem_before + mean_mem_after) / 2,
                    "Elapsed_Time": elapsed_time_total
                })

                print(f"\nQuantum Circuit for {num_qubits} Qubits, {reps_value} Layers, and {iterations} Iterations completed:")
                print(f"Elapsed Time: {elapsed_time_total} seconds")
                print(f"Mean CPU Usage: {(mean_cpu_before + mean_cpu_after) / 2}%")
                print(f"Mean Memory Usage: {(mean_mem_before + mean_mem_after) / 2}%")

print(f"Results saved in {csv_file_path}.")





Quantum Circuit for 2 Qubits, 1 Layers, and 10 Iterations:
     ┌──────────┐┌──────────┐     ┌──────────┐┌──────────┐
q_0: ┤ Ry(θ[0]) ├┤ Rz(θ[2]) ├──■──┤ Ry(θ[4]) ├┤ Rz(θ[6]) ├
     ├──────────┤├──────────┤┌─┴─┐├──────────┤├──────────┤
q_1: ┤ Ry(θ[1]) ├┤ Rz(θ[3]) ├┤ X ├┤ Ry(θ[5]) ├┤ Rz(θ[7]) ├
     └──────────┘└──────────┘└───┘└──────────┘└──────────┘

Quantum Circuit for 2 Qubits, 1 Layers, and 10 Iterations completed:
Elapsed Time: 35.37350296974182 seconds
Mean CPU Usage: 35.93%
Mean Memory Usage: 81.84%

Quantum Circuit for 2 Qubits, 1 Layers, and 50 Iterations:
     ┌──────────┐┌──────────┐     ┌──────────┐┌──────────┐
q_0: ┤ Ry(θ[0]) ├┤ Rz(θ[2]) ├──■──┤ Ry(θ[4]) ├┤ Rz(θ[6]) ├
     ├──────────┤├──────────┤┌─┴─┐├──────────┤├──────────┤
q_1: ┤ Ry(θ[1]) ├┤ Rz(θ[3]) ├┤ X ├┤ Ry(θ[5]) ├┤ Rz(θ[7]) ├
     └──────────┘└──────────┘└───┘└──────────┘└──────────┘

Quantum Circuit for 2 Qubits, 1 Layers, and 50 Iterations completed:
Elapsed Time: 30.329927921295166 seconds
Mean CPU Usage: 31.8